# Геокодирование адресов
> ##### Используются геокодеры Visicom и Nominatim

1. Создайте книгу excel, формат файла .xlsx
2. На первом листе книги, в ячейке 'A1' напишите имя заголовка
3. Начиная с ячейки A2 ... An. внесите без пропусков строк, адреса для геокодирования

>Желательный формат адреса: название населенного пункта (разделитель запятая) название области
> **Пример:** Анадоль, ДОНЕЦКАЯ

### Нажмите на кнопку и загрузите файл 

In [1]:
import ipywidgets as widgets

uploader = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
uploader

FileUpload(value={}, description='Upload')

### Предпросмотр загруженных адресов

In [22]:
import io
import openpyxl
from pandas import ExcelFile

excel_data = ExcelFile(io.BytesIO(uploader.data[-1]), engine='openpyxl')
dataframe = excel_data.parse(excel_data.sheet_names[0])
dataframe

,location
0,"Авдеевка, ДОНЕЦКАЯ"
1,"Азовское, ДОНЕЦКАЯ"
2,"Александровка, ДОНЕЦКАЯ"
3,"Алексеево-Дружковка, ДОНЕЦКАЯ"
4,"Анадоль, ДОНЕЦКАЯ"
5,"Андреевка, ДОНЕЦКАЯ"
6,"Андреевка селище міського типу, ДОНЕЦКАЯ"
7,"Андреевка село, ДОНЕЦКАЯ"
8,"Артема, ДОНЕЦКАЯ"
9,"Бабах-Тарама, ДОНЕЦКАЯ"


### Скрипт геокодирования для загруженного списка адресов

In [24]:
import os
import io
import re
import json
import time
import datetime

import pytz
from dateutil.tz import tzutc, tzlocal

import requests
from requests import cookies

import openpyxl
from pandas import ExcelFile

import geocoder
import ipysheet

from ipywidgets import HTML
from ipyleaflet import Map, Marker, Popup

center = (50.4547, 30.5238)
show_map = Map(center=center, zoom=9, close_popup_on_click=False)

GOOGLE_API_KEY = None
sheet = ipysheet.sheet(rows=len(dataframe.index), columns=4)
sheet.column_headers = ['Index', 'Address', 'lontitude', 'latitude']

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'}


class Epoch():

    def __init__(self):
        self.tzutc = tzutc()  # Время в UTC
        self.tzlocal = tzlocal()  # Локальное время
        self.winter_summer = time.daylight  # зимнее-летнее время
        self.__UNIX_EPOCH = datetime.datetime(1970, 1, 1, 0, 0,
                                              tzinfo=pytz.utc)
        self.complex_data = None

    def run(self):
        self.__year = self.complex_data[0]
        self.__month = self.complex_data[1]
        self.__day = self.complex_data[2]
        self.__hour = self.complex_data[3]
        self.__minute = self.complex_data[4]
        self.__second = self.complex_data[5]
        self.__microsecond = 0
        # Create local time
        naive = datetime.datetime(self.__year, self.__month,
                                  self.__day, self.__hour,
                                  self.__minute, self.__second,
                                  self.__microsecond,
                                  tzinfo=tzlocal())
        # Convert local time to UTC
        self.utc_datetime = naive.astimezone(pytz.utc)
        self.delta = self.utc_datetime - self.__UNIX_EPOCH
        self.seconds = self.delta.total_seconds()
        self.ms = self.seconds * 1000

        return self.seconds


def begin_get():
    start_UNIX_EPOCH = datetime.datetime(1970, 1, 1, 0, 0, tzinfo=pytz.utc)

    # Convert local time to UTC
    naive = datetime.datetime.now()
    utc_datetime = naive.astimezone(pytz.utc)
    delta = utc_datetime - start_UNIX_EPOCH
    seconds = delta.total_seconds()
    ms = int(seconds * 1000)

    # ---------------------------------
    # Вязть с сайта визиком из браузера F12, cookies
    ms = '1581407448242'
    # ---------------------------------

    jar = cookies.RequestsCookieJar()
    jar.set('timestamps', str(ms))
    jar.set('lang', 'ru')

    # --------------------------------------------------------------------------
    # Вязть с сайта визиком из браузера F12
    url = 'https://maps.visicom.ua/c/34.41184,47.49205,16/f/ADR3JRG6CVRUKHNPU3'
    # --------------------------------------------------------------------------

    payload = {'lang': 'ru'}

    ret = requests.get(url, headers=headers, params=payload, cookies=jar, timeout=60)
    assert ret.status_code == 200
    res = ret.content.decode('utf-8')
    # decode_json = json.loads(res)

    return ms, url


def geocode_visicom(adr, ms, ref):
    """
        Запрос к Visicom
    """
    decode_json = None
    name_error = None

    # 'Referer': 'https://maps.visicom.ua/c/34.41184,47.49205,16/f/ADR3JWI3J6MK7TW4UN?lang=ru'
    # start_UNIX_EPOCH = datetime.datetime(1970, 1, 1, 0, 0, tzinfo=pytz.utc)
    # Переводим локальное время в UTC
    # naive = datetime.datetime.now()
    # utc_datetime = naive.astimezone(pytz.utc)
    # delta = utc_datetime - start_UNIX_EPOCH
    # seconds = delta.total_seconds()
    # ms = int(seconds * 1000)

    url = 'https://maps.visicom.ua/ru/fts'
    payload = {'q': adr,
               'c': '34.37622,48.83218,7',
               'co': 'ua'}

    jar = cookies.RequestsCookieJar()
    jar.set('_ga', "GA1.2.1366087715.1580981745")
    jar.set('_gid', "GA1.2.1572761778.1580981745")
    jar.set('_gat', '1')
    jar.set('timestamp', str(ms))
    jar.set('lang', 'ru')

    headers_two = headers
    headers_two['Referer'] = ref
    err = "200 OK"

    try:
        ret = requests.get(url, headers=headers_two, params=payload, cookies=jar, timeout=60)
        assert ret.status_code == 200
        res = ret.content.decode('utf-8')
        decode_json = json.loads(res)

    except requests.ConnectionError as ex:
        name_error = 'Ошибка подключения: ' + str(ex)
    except requests.Timeout as ex:
        name_error = 'Ошибка, время ожидания истекло: ' + str(ex)
    except AssertionError as ex:
        name_error = 'Ошибка 403 (Forbidden)'
    except Exception as ex:
        print(str(ex))
    finally:
        return decode_json, name_error


def get_free_visicom(adr):
    ms, ref = begin_get()
    # adr = 'Украина, Вишневое, вул Дніпровська, 12'
    result, name_error = geocode_visicom(adr, ms, ref)

    # pprint.pprint(result)
    # print(name_error)

    return result


def extract_lat_long_google_address(address_or_zipcode):
    lat, lng = None, None
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    endpoint = f"{base_url}?address={address_or_zipcode}&key={GOOGLE_API_KEY}"
    r = requests.get(endpoint)
    if r.status_code not in range(200, 299):
        return None, None
    try:
        results = r.json()['results'][0]
        lat = results['geometry']['location']['lat']
        lng = results['geometry']['location']['lng']
    except Exception as ex:
        print(r.json()["error_message"])

    return lng, lat


def get_osm_lat_long(city):
    g = geocoder.osm(city)
    return g.json


def get_visicom_lat_long(raw_res):
    # Анализ полученных данных dict

    rs = None
    coordinates = None

    if raw_res:
        feat = raw_res['type']
        if 'Feature' == feat:
            feat = raw_res['properties']

            if 'settlement' in feat:
                rs = feat['settlement']

            if 'geo_centroid' in raw_res:
                geo_centroid = raw_res['geo_centroid']
                coordinates = geo_centroid['coordinates']
                # print(rs, coordinates)

        elif 'FeatureCollection' == feat:
            feat = raw_res['features']
            box = feat[0]
            prop = box['properties']
            if 'settlement' in prop:
                rs = prop['settlement']
                rsname = prop['name']
                rsid = prop['settlement_id']

            if 'geo_centroid' in box:
                geo_centroid = box['geo_centroid']
                coordinates = geo_centroid['coordinates']
                # print(rs, coordinates)

    return rs, coordinates


def add_marker_leaflet(jp, addr, lon, lat):
    
    global show_map
    
    marker = Marker(location=(lat, lon))
    show_map.add_layer(marker)
    
    message2 = HTML()
    message2.value = ', '.join([str(jp), addr])
    message2.placeholder = addr
#     message2.description = addr

    # Popup associated to a layer
    marker.popup = message2


def get_file_jupyter():

    global dataframe
    
    print()
    print('Index', 'Address', 'longtitude', 'latitude', sep='|')

    for index, row in dataframe.iterrows():
        yield [index, row]


def pull_jupyter(provider, current_provider, jpindex, jpdata):

    global sheet
    
    def additional_cells(jpval, addr, coord_lon, coord_lat):
        ipysheet.cell(jpval, 0, jpval)
        ipysheet.cell(jpval, 1, addr)
        ipysheet.cell(jpval, 2, coord_lon)
        ipysheet.cell(jpval, 3, coord_lat)
    
    res = {}
    coord_point = 'нет данных'
    location_address = jpdata['location']

    if current_provider == 'nominatim':
        res = get_osm_lat_long(location_address)

        if res:
            if 'lng' in res:
                lng = res['lng']
                lat = res['lat']
                coord_point = [lng, lat]
        else:
            current_provider = provider[1]

    if current_provider == 'visicom':
        raw_res = get_free_visicom(location_address)
        res, coordinates = get_visicom_lat_long(raw_res)
        coord_point = coordinates

    if current_provider == 'google':
        res = extract_lat_long_google_address(location_address)
        if res:
            lng = res[0]
            lat = res[1]
            coord_point = [lng, lat]

    if coord_point:
        
        additional_cells(jpindex, location_address, coord_point[0], coord_point[1])
        add_marker_leaflet(jpindex, location_address, coord_point[0], coord_point[1])
        
        print(jpindex, location_address, coord_point[0], coord_point[1], sep=' | ')
    else:
        additional_cells(jpindex, location_address, coord_point, coord_point)
        print(jpindex, location_address, coord_point, coord_point, sep=' | ')

    time.sleep(1)


if __name__ == "__main__":

    provider = ('visicom', 'nominatim', 'google')

    for b in get_file_jupyter():
        pull_jupyter(provider, provider[0], b[0], b[1])


Index|Address|longtitude|latitude
0 | Авдеевка, ДОНЕЦКАЯ | 37.739009 | 48.14797
1 | Азовское, ДОНЕЦКАЯ | 37.341092 | 46.956598
2 | Александровка, ДОНЕЦКАЯ | 36.915124 | 48.707313
3 | Алексеево-Дружковка, ДОНЕЦКАЯ | 37.609543 | 48.580635
4 | Анадоль, ДОНЕЦКАЯ | 37.579244 | 47.454547
5 | Андреевка, ДОНЕЦКАЯ | 38.725987 | 48.109523
6 | Андреевка селище міського типу, ДОНЕЦКАЯ | None | None
7 | Андреевка село, ДОНЕЦКАЯ | 38.128923 | 47.826714
8 | Артема, ДОНЕЦКАЯ | 37.788982 | 48.031164
9 | Бабах-Тарама, ДОНЕЦКАЯ | 37.085931 | 46.896651


**Выделите данные и скопируйте через Ctrl+C**

In [21]:
sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='numeric', value=0), Cell(column_…

# Отображение координат на карте
>
> для просмотра адреса нажмите на маркер

In [19]:
show_map

Map(center=[50.4547, 30.5238], close_popup_on_click=False, controls=(ZoomControl(options=['position', 'zoom_in…